Set path to where `mlruns` directory is located (usually, the `CardiacCOMA` repository)

In [1]:
CARDIAC_COMA_REPO = "/home/rodrigo/CISTIB/repos/CardiacCOMA/"

In [2]:
import mlflow
import os, sys

import torch
import torch.nn.functional as F

import os; os.chdir(CARDIAC_COMA_REPO)
from config.load_config import load_yaml_config, to_dict

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import Image
from mlflow.tracking import MlflowClient

import pickle as pkl
import pytorch_lightning as pl

from argparse import Namespace
import matplotlib.pyplot as plt

#import surgeon_pytorch
#from surgeon_pytorch import Inspect, get_layers

import numpy as np
from IPython import embed
sys.path.insert(0, '..')

import model.Model3D
from utils.helpers import get_coma_args, get_lightning_module, get_datamodule
from copy import deepcopy
from pprint import pprint

In [3]:
import random
import pyvista as pv
import ast
import pandas as pd

Some function definitions:

In [4]:
def mse(s1, s2=None):
    if s2 is None:
        s2 = torch.zeros_like(s1)
    return ((s1-s2)**2).sum(-1).mean(-1)

In [5]:
client = MlflowClient()
local_dir = "/tmp/artifact_downloads"

if not os.path.exists(local_dir):
    os.mkdir(local_dir)
    
# client._tracking_client.list_artifacts(
#     runs_df.run_id[2], 'restored_model_checkp# oint'
# )

# Select MLflow experiment

In [6]:
TRACKING_URI = f"file://{CARDIAC_COMA_REPO}/mlruns"
mlflow.set_tracking_uri(TRACKING_URI)

In [7]:
options = [exp.name for exp in mlflow.list_experiments()]

experiment_w = widgets.Select(
    options=options,
    value=options[0]
)
display(experiment_w)

Select(options=('Cardiac - ED', 'Default'), value='Cardiac - ED')

Retrieve run data from MLflow

In [12]:
# runs_list = mlflow.search_runs(experiment_ids=[exp_id], output_format="list")
exp_id = mlflow.get_experiment_by_name(experiment_w.value).experiment_id

runs_df = mlflow.search_runs(experiment_ids=[exp_id],)

# Keep only the runs that ended successfully
runs_df = runs_df[runs_df.status == "FINISHED"].reset_index(drop=True)

# Use experiment ID and run ID as indices
runs_df = runs_df.set_index(["experiment_id", "run_id"])
# runs_df.describe()

Choose the run with the minimum `val_recon_loss` and load pretained weights.

In [13]:
runs_df = runs_df[(runs_df["params.latent_dim"] == "8")]# & (runs_df["metrics.val_recon_loss"] < 2 )]
print(runs_df.shape)
runs_df.head()

(41, 51)


status  \
experiment_id run_id                                       
1             2773a8ba02e84cb29fe7a692fff66fc6  FINISHED   
              8d082d60e7784e3e9b2bd620424225e8  FINISHED   
              2c83b43cbb314a9795911d3981360067  FINISHED   
              3ccf1e5e892949878a1ceba478de3457  FINISHED   
              346c47ff24a5456c8b77d7d0e34a0a9e  FINISHED   

                                                                                     artifact_uri  \
experiment_id run_id                                                                                
1             2773a8ba02e84cb29fe7a692fff66fc6  file:///home/rodrigo/CISTIB/repos/CardiacCOMA/...   
              8d082d60e7784e3e9b2bd620424225e8  file:///home/rodrigo/CISTIB/repos/CardiacCOMA/...   
              2c83b43cbb314a9795911d3981360067  file:///home/rodrigo/CISTIB/repos/CardiacCOMA/...   
              3ccf1e5e892949878a1ceba478de3457  file:///home/rodrigo/CISTIB/repos/CardiacCOMA/...   
              346c47ff24a5456c8b77d7d0e34a0a9e  file:///home/rodrigo/CISTIB/repos/CardiacCOMA/...   

                                                                     start_time  \
experiment_id run_id                                                              
1             2773a8ba02e84cb29fe7a692fff66fc6 2022-07-07 13:33:41.756000+00:00   
              8d082d60e7784e3e9b2bd620424225e8 2022-07-07 13:33:22.301000+00:00   
              2c83b43cbb314a9795911d3981360067 2022-07-07 08:57:25.726000+00:00   
              3ccf1e5e892949878a1ceba478de3457 2022-07-07 08:57:12.057000+00:00   
              346c47ff24a5456c8b77d7d0e34a0a9e 2022-07-07 08:57:02.179000+00:00   

                                                                       end_time  \
experiment_id run_id                                                              
1             2773a8ba02e84cb29fe7a692fff66fc6 2022-07-07 14:52:42.951000+00:00   
              8d082d60e7784e3e9b2bd620424225e8 2022-07-08 02:41:15.604000+00:00   
              2c83b43cbb314a9795911d3981360067 2022-07-07 12:46:52.206000+00:00   
              3ccf1e5e892949878a1ceba478de3457 2022-07-07 10:06:26.327000+00:00   
              346c47ff24a5456c8b77d7d0e34a0a9e 2022-07-07 12:49:58.855000+00:00   

                                                metrics.val_kld_loss  \
experiment_id run_id                                                   
1             2773a8ba02e84cb29fe7a692fff66fc6                   inf   
              8d082d60e7784e3e9b2bd620424225e8              3.741764   
              2c83b43cbb314a9795911d3981360067              4.752035   
              3ccf1e5e892949878a1ceba478de3457            187.436127   
              346c47ff24a5456c8b77d7d0e34a0a9e             29.892481   

                                                metrics.recon_loss  \
experiment_id run_id                                                 
1             2773a8ba02e84cb29fe7a692fff66fc6        1.560820e+07   
              8d082d60e7784e3e9b2bd620424225e8        4.065629e-01   
              2c83b43cbb314a9795911d3981360067        6.754199e-01   
              3ccf1e5e892949878a1ceba478de3457        1.662557e+00   
              346c47ff24a5456c8b77d7d0e34a0a9e        1.225944e+00   

                                                metrics.kld_loss  \
experiment_id run_id                                               
1             2773a8ba02e84cb29fe7a692fff66fc6               inf   
              8d082d60e7784e3e9b2bd620424225e8          3.749800   
              2c83b43cbb314a9795911d3981360067          4.733474   
              3ccf1e5e892949878a1ceba478de3457        187.487762   
              346c47ff24a5456c8b77d7d0e34a0a9e         29.888357   

                                                metrics.epoch  metrics.loss  \
experiment_id run_id                                                          
1             2773a8ba02e84cb29fe7a692fff66fc6           20.0           inf   
              8d082d60e7784e3e9b2bd620424225e8 

In [14]:
runs_df["params.latent_dim"]

experiment_id  run_id                          
1              2773a8ba02e84cb29fe7a692fff66fc6    8
               8d082d60e7784e3e9b2bd620424225e8    8
               2c83b43cbb314a9795911d3981360067    8
               3ccf1e5e892949878a1ceba478de3457    8
               346c47ff24a5456c8b77d7d0e34a0a9e    8
               24b8a4654f1f483bbf46558d3ecdf8d5    8
               943303879aab4e5ba18877d48a8bc229    8
               22b8553e562b406e8ea63362582665ea    8
               26c8a3565a32442399619242ae657220    8
               3cce94de67054ec5890958b3419f3c99    8
               979ba75551db4eee9bbf6572f31ac7b9    8
               f85bb684cf114057bb0dbdf9dc1a7b95    8
               7449da4588ae4099a4f06719269d5aba    8
               4eedb101561b43abbf28e44fe0500265    8
               71a4ff573871415f9f6320ab3e0470fc    8
               b0d1636c1e184107a12bbdad6be40e12    8
               2b7fddec29474c62a72805723eabe062    8
               adefb34340f7430ba79309d33c965b1b    

In [15]:
run_ids_w = widgets.Select(options=[x[0] + "-" + x[1] for x in runs_df.index])
display(run_ids_w)

Select(options=('1-2773a8ba02e84cb29fe7a692fff66fc6', '1-8d082d60e7784e3e9b2bd620424225e8', '1-2c83b43cbb314a9…

In [16]:
experiment_id, run_id = runs_df.index[runs_df["metrics.val_recon_loss"].argmin()]
experiment_id, run_id = run_ids_w.value.split("-")
print(experiment_id, run_id)
# print(runs_df.loc[experiment_id, run_id].to_dict())

run_info = runs_df.loc[experiment_id, run_id].to_dict()

# artifact_uri = run_info["artifact_uri"].replace("file://", "")
chkpt_dir = os.path.join(TRACKING_URI.replace("file://", ""), experiment_id, run_id, "checkpoints")
# chkpt_dir = os.path.join(artifact_uri, "restored_model_checkpoint")
chkpt_file = os.path.join(chkpt_dir, os.listdir(chkpt_dir)[0])

model_pretrained_weights = torch.load(chkpt_file, map_location=torch.device('cpu'))["state_dict"]

# Remove "model." prefix from state_dict's keys.
_model_pretrained_weights = {k.replace("model.", ""): v for k, v in model_pretrained_weights.items()}
# print(_model_pretrained_weights)

1 2c83b43cbb314a9795911d3981360067


In [17]:
def overwrite_ref_config(ref_config, run_info):
    
    '''
    This is a workaround for adjusting the configuration of those runs that didn't have a YAML configuration file logged as an artifact.
    '''
    
    config = deepcopy(ref_config)
    config.network_architecture.latent_dim = int(run_info["params.latent_dim"])
    config.loss.regularization.weight = float(run_info["params.w_kl"])
    config.optimizer.parameters.lr = float(run_info["params.lr"])
    config.network_architecture.convolution.channels_enc = [int(x) for x in ast.literal_eval(run_info["params.n_channels_enc"])]
    config.network_architecture.convolution.channels_dec = [int(x) for x in ast.literal_eval(run_info["params.n_channels_enc"])]
    config.sample_sizes = [100, 100, 100, -1]
    
    return config


ref_config = load_yaml_config("config_files/config.yaml")
config = overwrite_ref_config(ref_config, run_info)
# pprint(to_dict(config))

In [18]:
dm = get_datamodule(config, perform_setup=True)
ids = list(dm.dataset_args['meshes'].keys())
meshes = dm.dataset_args['meshes']
model = get_lightning_module(config, dm)
model.model.load_state_dict(_model_pretrained_weights)
model.model.mode = "testing"

[100, 100, 100, -1]


In [ ]:
s = dm.dataset[ids[0]]['s']
s_hat = model(s)[0][0]
mse(s, s_hat)

# Display meshes for subjects

In [38]:
pv.set_plot_theme("document")

faces, _ = pkl.load(open("data/cardio/faces_and_downsampling_mtx_frac_0.1_LV.pkl", "rb")).values()
faces = np.c_[np.ones(faces.shape[0]) * 3, faces].astype(int)

color_palette = list(pv.colors.color_names.values())
random.shuffle(color_palette)


def f(id):                
    pl = pv.Plotter(notebook=True, off_screen=False, polygon_smoothing=False)
    mesh = meshes[id]
    mesh = pv.PolyData(mesh, faces)    
    pl.add_mesh(mesh, show_edges=False, point_size=1.5, color=color_palette[0], opacity=0.5)
    pl.show(interactive=False, interactive_update=False)
    
interact(f, 
  id=widgets.Select(options=ids)
  #rotated=widgets.ToggleButton(),
  #traslated=widgets.ToggleButton()
);

interactive(children=(Select(description='id', options=('4705339', '3040882', '2653145', '4041510', '4793598',…

## Generate synthetic shapes

In [17]:
#def apply_deformation(, spherical_coords, amplitude_lm):
#    
#    deformed_sphere = sphere
#    
#    coefs = np.ones(spherical_coords.shape[0])
#    
#    for l, m in amplitude_lm:        
#        coefs += amplitude_lm[l, m] * Y_lm[l,m]
#                
#    deformed_sphere.vertices = np.multiply(
#        deformed_sphere.vertices, np.kron(np.ones((3, 1)), coefs).transpose()
#    )
#    
#    return deformed_sphere



In [19]:
z_df = pd.read_csv(f"{CARDIAC_COMA_REPO}/mlruns/{experiment_id}/{run_id}/artifacts/output/latent_vector.csv")
z_mean = z_df[z_df.columns[z_df.columns.str.startswith("z")]].mean()
z_std = z_df[z_df.columns[z_df.columns.str.startswith("z")]].std()

In [20]:
pv.set_plot_theme("document")

faces, _ = pkl.load(open("data/cardio/faces_and_downsampling_mtx_frac_0.1_LV.pkl", "rb")).values()
faces = np.c_[np.ones(faces.shape[0]) * 3, faces].astype(int)

color_palette = list(pv.colors.color_names.values())
random.shuffle(color_palette)

def get_coef_as_string(n):
    return "z" + str(n).zfill(3)

In [21]:
def f(**kwargs):

    coefs = np.array([kwargs[lm] for lm in kwargs])
    z = z_mean + coefs * np.array(z_std)
    z = torch.Tensor(z)
    with torch.no_grad():
        point_cloud = model.model.decoder(z).detach().numpy()[0]
    # spherical_coords = appendSpherical_np(sphere.vertices)[:,3:]
    # deformed_sphere = apply_deformation(sphere, spherical_coords, coefs)
                
    mesh = pv.PolyData(point_cloud, faces)
    plotter = pv.Plotter(notebook=True, off_screen=False, polygon_smoothing=False)
    plotter.add_mesh(mesh, show_edges=False, point_size=1.5, color=color_palette[0], opacity=0.5)
    plotter.show(interactive=False, interactive_update=False)

In [22]:
coefs = {get_coef_as_string(n): widgets.FloatSlider(min=-3, max=3) for n in range(8)}
interact(f, **coefs);

interactive(children=(FloatSlider(value=0.0, description='z000', max=3.0, min=-3.0), FloatSlider(value=0.0, de…